In [22]:
import pandas as pd 
from sklearn.model_selection import train_test_split
df = pd.read_excel('FineTuningData/fine_tune_dataset.xlsx').fillna('')
df[6] = df[6].astype(str)
X = df[6].to_list(); y = df['label'].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(X, 
                                                                      y, 
                                                                      random_state=42, 
                                                                      stratify=y, 
                                                                      test_size=.20)

# print(len(train_texts), len(train_labels), len(test_texts), len(test_labels))

train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, 
                                                                        train_labels, 
                                                                        random_state=42, 
                                                                        stratify=train_labels, 
                                                                        test_size=.10)


In [33]:
from transformers import RobertaTokenizerFast
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
tokenizer = RobertaTokenizerFast.from_pretrained("./LSCP_tokenizer", max_len=256)
model = RobertaForSequenceClassification.from_pretrained("./LSCP/")

Some weights of the model checkpoint at ./LSCP/ were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./LSCP/ and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', '

In [23]:
# print(train_texts[:10])
for item in train_texts:
    if isinstance(item, str)==False :
        print('youp!')
# print(valid_texts[:10])
# print(test_texts[:10])


In [34]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(valid_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [35]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(val_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [60]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=30,  # batch size per device during training
    per_device_eval_batch_size=30,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
#     evaluation_strategy  = 'epoch'
    
)
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,             # evaluation dataset
#     evaluation_strategy='epoch',
)

trainer.train()

You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


TrainOutput(global_step=3600, training_loss=0.008797277343647289)

In [61]:
pred = trainer.predict(test_dataset)

In [62]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [63]:
compute_metrics(pred)

{'accuracy': 0.9522935779816514,
 'f1': 0.9501915708812261,
 'precision': 0.9508539560822586,
 'recall': 0.9495301079011487}

In [58]:
trainer.evaluate()

/home/textmining/anaconda3/envs/persian_bert/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.31366131457888213, 'epoch': 5.447222222222222}